In [1]:
!pip install --no-cache-dir transformers sentencepiece

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 48.7 MB/s 
     |████████████████████████████████| 895 kB 44.3 MB/s 
     |████████████████████████████████| 3.3 MB 33.0 MB/s 
     |████████████████████████████████| 596 kB 43.2 MB/s 
     |████████████████████████████████| 61 kB 11.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pip install datasets sacrebleu

     |████████████████████████████████| 298 kB 5.4 MB/s 
     |████████████████████████████████| 90 kB 9.3 MB/s 
     |████████████████████████████████| 243 kB 43.9 MB/s 
     |████████████████████████████████| 132 kB 46.3 MB/s 
     |████████████████████████████████| 1.1 MB 33.3 MB/s 
     |████████████████████████████████| 160 kB 43.0 MB/s 
     |████████████████████████████████| 271 kB 51.9 MB/s 
     |████████████████████████████████| 192 kB 55.4 MB/s 


We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using the Trainer API.

In [3]:
model_checkpoint = "google/mt5-small"

In [4]:
from datasets import Dataset, load_metric, load_dataset

metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

In [5]:
import torch
import numpy as np
import pandas as pd
import os
from google.colab import drive
import logging

In [6]:
drive.mount('content')

Mounted at content


In [7]:
PATH_TO_DATASET = "content/MyDrive"

In [8]:
data  =  pd.read_excel(os.path.join(PATH_TO_DATASET, "Data.xlsx"), index_col = 0)

In [9]:
data.head()

,Arabic_transcript,English_transcript
0,مرحبا بكم في مقدمة لعلوم البيانات مع بايثون. ه...,Welcome to an introduction to Data Science wit...
1,مرحباً، أنا (كريس بروكز)، هيئة التدريس هنا بكل...,"Hi, I'm Chris Brooks, faculty here at the Univ..."
2,مرحبا. أريد أن أريكم قليلا عن نظام دفتر جوبيتر...,Hi. I want to show you a little bit about the ...
3,في بقية هذه الوحدة، سأقوم بتقديم نظرة عامة أسا...,"In the rest of this module, I'm going to provi..."
5,تحدثنا عن السلاسل عندما تحدثنا عن القوائم والت...,We talked about strings when we talked about l...


In [10]:
data.shape

(16818, 2)

In [11]:
import random

random.seed(10)

train_set, validate_set, test_set = np.split(data.sample(frac=1), [int(.8*len(data)), int(.9*len(data))])
# https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn 



In [12]:
train_set.shape, validate_set.shape, test_set.shape

((13454, 2), (1682, 2), (1682, 2))

In [13]:
train_set.isnull().sum() / train_set.shape[0] *100

Arabic_transcript     0.044596
English_transcript    0.000000
dtype: float64

In [14]:
train_set.dropna(inplace=True)
validate_set.dropna(inplace=True)

In [15]:
train_set.isnull().sum() / train_set.shape[0] *100

Arabic_transcript     0.0
English_transcript    0.0
dtype: float64

In [16]:
train_set = train_set.drop_duplicates(subset=['Arabic_transcript', 'English_transcript'])

# train_set.head()
train_set['English_transcript'] = train_set.apply(lambda row: row.English_transcript.lower(), axis=1)
train_set.columns = ['ar', 'en']

In [17]:
validate_set = validate_set.drop_duplicates(subset=['Arabic_transcript', 'English_transcript'])

# train_set.head()
validate_set['English_transcript'] = validate_set.apply(lambda row: row.English_transcript.lower(), axis=1)
validate_set.columns = ['ar', 'en']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


text generated seems to be not random. I have got something wrong here

### Pre-processing for arabic text?

In [18]:
train_set.head()

,ar,en
4198,اممم ، و- وسوف ترى x عندما ننتقل إلى GLMs.,"um, and- and you're gonna see x when we move o..."
8368,إذن فهو متغير عشوائي.,so it's a random variable.
6279,لكن لذلك ، فإن نمذجة هذا بدون افتراضات إضافية ...,"but so, modeling this without additional assum..."
15958,وهو نوع من الخبز,"which is a kind of a bread and butter of,"
11735,سيكون الأمر ممتع.,it's going to be fun.


**Text cleaning?**

In [19]:
pip install mpu

     |████████████████████████████████| 69 kB 3.5 MB/s 


In [20]:
train_set['translation'] = train_set.apply(lambda row: {'en':row['en'], 'ar':row['ar']},
                            axis=1)
train_set = train_set.drop(columns = ['ar', 'en'])  #https://stackoverflow.com/questions/55136065/convert-multiple-pandas-column-into-json 
train_set.head()

,translation
4198,"{'en': 'um, and- and you're gonna see x when w..."
8368,"{'en': 'so it's a random variable.', 'ar': 'إذ..."
6279,"{'en': 'but so, modeling this without addition..."
15958,{'en': 'which is a kind of a bread and butter ...
11735,"{'en': 'it's going to be fun.', 'ar': 'سيكون ا..."


In [21]:
validate_set['translation'] = validate_set.apply(lambda row: {'en':row['en'], 'ar':row['ar']},
                            axis=1)
validate_set = validate_set.drop(columns = ['ar', 'en'])  #https://stackoverflow.com/questions/55136065/convert-multiple-pandas-column-into-json 
validate_set.head()

,translation
8059,{'en': 'my actions are going to be stay or qui...
11813,"{'en': 'uh, objects from big,', 'ar': 'آه ، أش..."
4286,"{'en': 'so, uh, a bernoulli distribution is on..."
13586,{'en': 'supervisees is for them to think about...
6239,{'en': 'we'll- we- we'll get cs229 there somed...


In [22]:
import json

import mpu.io
# https://www.py4u.net/discuss/191017 
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_json.html


with open('data.json', 'w', encoding='utf-8') as file:
    train_set.to_json(file, orient="records", force_ascii=False, lines = True)

translation = load_dataset('json', data_files= 'data.json')
# https://huggingface.co/docs/datasets/loading_datasets.html#json-files 

Using custom data configuration default-e0a5b92b2331a807


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e0a5b92b2331a807/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
with open('data_val.json', 'w', encoding='utf-8') as file:
    validate_set.to_json(file, orient="records", force_ascii=False, lines = True)

translation_val = load_dataset('json', data_files= 'data_val.json')
# https://huggingface.co/docs/datasets/loading_datasets.html#json-files 

Using custom data configuration default-8b5be33cc942c16b


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-8b5be33cc942c16b/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
translation

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 13309
    })
})

In [25]:
translation_val

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1676
    })
})

# Fine tuning 
I will be using 'data' dataset. It contains (2 coursera courses, springer file, DS_codata_org, yt_stanford). 

I will mainly depend on this Nb from huggingface notebooks
https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb 

In [26]:
cuda = torch.device('cuda') 

### Create custom Datasets Class

## Preprocessing the data

In [50]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)  #https://discuss.huggingface.co/t/error-with-new-tokenizers-urgent/2847/4

loading configuration file https://huggingface.co/google/mt5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/97693496c1a0cae463bd18428187f9e9924d2dfbadaa46e4d468634a0fc95a41.dadce13f8f85f4825168354a04675d4b177749f8f11b167e87676777695d4fe4
Model config MT5Config {
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.12.5",
  "use_cache": true,
  "vocab_size": 250112
}

loading file https://huggingface.co/google/mt5-small/resolve/main/spiece.model from cache at

In [51]:
if "t5" in model_checkpoint:
    tokenizer.src_lang = "en_XX"
    tokenizer.tgt_lang = "ar_AR"  
    tokenizer.source_prefix = "translate English to Arabic: "

In [55]:
tokenizer("Hello, this one sentence!")

{'input_ids': [30273, 261, 714, 1371, 259, 98923, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [56]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[30273, 261, 714, 1371, 259, 98923, 309, 1], [1494, 339, 259, 7845, 259, 98923, 260, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [57]:
# if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
#     prefix = "translate English to Arabic: "
# else:
#     prefix = ""

since we are using mt5 small, I will add this manually

In [58]:
prefix = "translate English to Arabic: "

In [59]:
model_checkpoint

'google/mt5-small'

In [60]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ar"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [61]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [62]:
show_random_elements(translation["train"])

In [63]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: The system stream (a sequence of segments).
    references: A list of one or more reference streams (each a sequence of segments).
    smooth_method: The smoothing method to use. (Default: 'exp').
    smooth_value: The smoothing value. Only valid for 'floor' and 'add-k'. (Defaults: floor: 0.1, add-k: 1).
    tokenize: Tokenization method to use for BLEU. If not provided, defaults to 'zh' for Chinese, 'ja-mecab' for
        Japanese and '13a' (mteval) otherwise.
    lowercase: Lowercase the data. If True, enables case-insensitivity. (Default: False).
    force: Insist that your tokenized input is actually detokenized.

Returns:
    'score': BLEU score,
    'counts'

### Tokenization

In [64]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/google/mt5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/97693496c1a0cae463bd18428187f9e9924d2dfbadaa46e4d468634a0fc95a41.dadce13f8f85f4825168354a04675d4b177749f8f11b167e87676777695d4fe4
Model config MT5Config {
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.12.5",
  "use_cache": true,
  "vocab_size": 250112
}

loading file https://huggingface.co/google/mt5-small/resolve/main/spiece.model from cache at

To apply this function on all the pairs of sentences in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command.

In [68]:
tokenized_datasets = translation.map(preprocess_function, batched=True)

  0%|          | 0/14 [00:00<?, ?ba/s]

In [69]:
tokenized_datasets_val = translation_val.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

##Fine-tuning the model


In [67]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/google/mt5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/97693496c1a0cae463bd18428187f9e9924d2dfbadaa46e4d468634a0fc95a41.dadce13f8f85f4825168354a04675d4b177749f8f11b167e87676777695d4fe4
Model config MT5Config {
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.12.5",
  "use_cache": true,
  "vocab_size": 250112
}

loading weights file https://huggingface.co/google/mt5-small/resolve/main/pytorch_model.bin 

In [70]:
batch_size = 4
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False, # to avoid nan error but this will make training much slower
    # push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [71]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

we have to do a bit of pre-processing to decode the predictions into texts:

In [72]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [73]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets_val["train"], # this is validation, But I couldn't create a dataset that is composed of training and validation. 
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [74]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 13309
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3328


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,4.431600,2.914065,0.166300,10.897400


Saving model checkpoint to mt5-small-finetuned-en-to-ar/checkpoint-500
Configuration saved in mt5-small-finetuned-en-to-ar/checkpoint-500/config.json
Model weights saved in mt5-small-finetuned-en-to-ar/checkpoint-500/pytorch_model.bin
tokenizer config file saved in mt5-small-finetuned-en-to-ar/checkpoint-500/tokenizer_config.json
Special tokens file saved in mt5-small-finetuned-en-to-ar/checkpoint-500/special_tokens_map.json
Copy vocab file to mt5-small-finetuned-en-to-ar/checkpoint-500/spiece.model
Deleting older checkpoint [mt5-small-finetuned-en-to-ar/checkpoint-2000] due to args.save_total_limit
Saving model checkpoint to mt5-small-finetuned-en-to-ar/checkpoint-1000
Configuration saved in mt5-small-finetuned-en-to-ar/checkpoint-1000/config.json
Model weights saved in mt5-small-finetuned-en-to-ar/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in mt5-small-finetuned-en-to-ar/checkpoint-1000/tokenizer_config.json
Special tokens file saved in mt5-small-finetuned-en-to-ar

TrainOutput(global_step=3328, training_loss=6.73314303618211, metrics={'train_runtime': 2347.8879, 'train_samples_per_second': 5.668, 'train_steps_per_second': 1.417, 'total_flos': 1181572768696320.0, 'train_loss': 6.73314303618211, 'epoch': 1.0})